<a href="https://colab.research.google.com/github/trefftzc/trefftzc/blob/main/Using_mpi4py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring mpi4py on COLAB

Install mpi4py

In [1]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746506 sha256=ed0b5dbd517a38ac522f1bee8b075b95c65d69529471ef67070b8d2479df758f
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


Create a file in a cell and then run it.

In [2]:
%%writefile example.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1, tag=11)
elif rank == 1:
    data = comm.recv(source=0, tag=11)

Writing example.py


In [3]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python example.py

Solving the partition problem with MPI.

In [9]:
%%writefile partition_mpi.py

from mpi4py import MPI

def evaluatePartition(  value,  n, array) :
   sum0s = 0
   sum1s = 0
   mask = 1
   for i in range(0,n):
     if ((mask & value) != 0):
       sum1s = sum1s + array[i]
     else:
       sum0s = sum0s + array[i]
     mask = mask * 2

   if (sum0s == sum1s):
     return 1
   else:
     return 0

def printResults(value, n, array):
  print("Solution:\n")
  print("First partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) != 0):
      print(array[i]," ")
      sum = sum + array[i]
    mask = mask * 2

  print(" sum: ",sum)
  print("Second partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) == 0):
      print(array[i]," ")
      sum = sum + array[i]
    mask = mask * 2
  print(" sum: \n",sum)

if __name__ == "__main__":
  comm = MPI.COMM_WORLD
  rank = comm.Get_rank()
  size = comm.Get_size()
  if rank == 0:
    array = [1,2,3,4,5,5]
  else:
    array = None
# The collective operation broadcast
# sends a particular object to all other nodes
  array = comm.bcast(array, root=0)

  n = len(array)
  n_partitions = 1
  for i in range(1,n):
    n_partitions = n_partitions * 2

  # Now divide the total number of n_partitions
  # by the number of participating nodes
  chunk_size = n_partitions // size

  #
  # Every node works on a different portion of the range
  #
  start = rank * chunk_size
  if rank == (size-1):
    end = n_partitions
  else:
    end = start + chunk_size
  solutionFound = 0
  solution = -1
  for i in range(start,end):
    if (evaluatePartition(i,n,array) == 1):
      solutionFound = 1
      solution = i
      break
 #
 # Now perform a reduction
 # The result will be place in the root node
 #
  solution = comm.reduce(solution,op=MPI.MAX,root=0)

  if rank == 0:
    if (solution != -1):
      printResults(solution,n,array)
    else:
      print("No solution found for this instance")



Overwriting partition_mpi.py


Now execute the program.

In [10]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python partition_mpi.py

Solution:

First partition: 
2  
3  
5  
 sum:  10
Second partition: 
1  
4  
5  
 sum: 
 10
